ImportError: attempted relative import with no known parent package

In [ ]:
for epoch in range(100):
    equalization_loss, distil_loss, target_loss = dc(sub), dc(sub), dc(sub)
    equal_loss_adv, distil_loss_adv, match3 = dc(sub), dc(sub), dc(sub)
    optimizer.zero_grad()

    new_z = target_model.forward_(wz_feature, att_mask)
    z_tar = new_z[:target_len]
    z_mix = new_z[target_len + bias_len:]

    # for wz_, z0_ in zip(z_tar, z0_tar):
    #     distil_loss += F.mse_loss(wz_, z0_).to(device) * 0.0005
    # for wz_, z0_ in zip(z_mix, z0_mix):
    #     distil_loss += F.mse_loss(wz_, z0_).to(device)
    z_tar_eos = F.normalize(z_tar[torch.arange(target_len), eos_token_ind[:target_len]], dim=-1)
    z_mix_eos = F.normalize(z_mix[torch.arange(mix_len), eos_token_ind[target_len + bias_len:]], dim=-1)


    sim_tar_mix0 = (z_tar_eos @ z0_mix_eos.T)  # doctor and mix
    sim_tar_bias0 = (z_tar_eos @ z0_bias_eos.T)  # doctor and bias
    sim_mix_bias0_0 = (z_mix_eos[:bias_len] @ z0_bias_eos.T)  # mix and bias
    sim_mix_bias0_1 = (z_mix_eos[bias_len:] @ z0_bias_eos.T)  # mix and bias
    sim_mix_mix = (z_mix_eos[:bias_len] @ z_mix_eos[bias_len:].T)
    match = torch.relu(sim_mix_bias0_0[~ind_].max() - sim_mix_bias0_0[ind_].min() + 0.1) + torch.relu(
        sim_mix_bias0_1[~ind_].max() - sim_mix_bias0_1[ind_].min() + 0.1) + torch.relu(
        sim_mix_mix[~ind_].max() - sim_mix_mix[ind_].min() + 0.1)
    match2 = torch.relu(sim_tar_mix0[indices_3].max() - sim_tar_mix0[indices_1].min() + 0.15) + torch.relu(
        sim_tar_mix0[indices_3].max() - sim_tar_mix0[indices_2].min() + 0.15)

    # wrong_thing_ = torch.cat([sim_mix_mix[~ind_], sim_tar_mix0[indices_3]])
    # right_thing_ = torch.cat([sim_mix_mix[ind_], sim_tar_mix0[~indices_3]])
    # match3 += abs(max(wrong_thing_) - min(right_thing_)+0.1)
    # wrong_thing = torch.cat([sim_mix_bias0_0[~ind_], sim_mix_bias0_1[~ind_]])
    # right_thing = torch.cat([sim_mix_bias0_0[ind_], sim_mix_bias0_1[ind_]])
    # match3 += abs(max(wrong_thing) - min(right_thing) +0.1)
    # print('wrong_thing : ', (wrong_thing_ * 100).int().tolist(), 'right_thing : ', (right_thing_ * 100).int().tolist(),
    #       'wrong_thing : ', (wrong_thing * 100).int().tolist(), 'right_thing : ', (right_thing * 100).int().tolist())

    sim_mix_mix_right = (z_mix_eos[:bias_len] @ z_mix_eos[bias_len:].T)[ind_]
    sim_mix0_mix0_wrong = (z_mix_eos[:bias_len] @ z_mix_eos[bias_len:].T)[~ind_]
    # tar - mix
    sim_tar0_mix0_right = (z_tar_eos @ z_mix_eos.T)[~indices_3]
    sim_tar0_mix0_wrong = (z_tar_eos @ z_mix_eos.T)[indices_3]
    # bias
    sim_tar0_bias0 = z_tar_eos @ z0_bias_eos.T
    # mix - bias
    sim_mix0_bias0_right = (z_mix_eos @ z0_bias_eos.T)[ind_2]
    sim_mix0_bias0_wrong = (z_mix_eos @ z0_bias_eos.T)[~ind_2]

    print(f'ck_list : mix_r {int(sim_mix_mix_right.mean() * 100)}, w {int(sim_mix0_mix0_wrong.mean() * 100)}'
          f'| tar_mix_r {int(sim_tar0_mix0_right.mean() * 100)}, w {int(sim_tar0_mix0_wrong.mean() * 100)}'
          f'| mix_bias_r {int(sim_mix0_bias0_right.mean() * 100)}, w {int(sim_mix0_bias0_wrong.mean() * 100)}'
          f'| tar_bias {int(sim_tar0_bias0.mean() * 100)}')

    sim_tar_mix0_wrong = (z_tar_eos @ z0_mix_eos.T)[indices_3]
    sim_tar0_mix_wrong = (z0_tar_eos @ z_mix_eos.T)[indices_3]
    sim_tar_mix0_right = (z_tar_eos @ z0_mix_eos.T)[~indices_3]
    sim_tar0_mix_right = (z0_tar_eos @ z_mix_eos.T)[~indices_3]
    sim_tar_mix_right = (z_tar_eos @ z_mix_eos.T)[~indices_3]
    sim_tar_bias0 = (z_tar_eos @ z0_bias_eos.T)
    sim_mix_bias0_0 = (z_mix_eos[:bias_len] @ z0_bias_eos.T)
    sim_mix_bias0_1 = (z_mix_eos[bias_len:] @ z0_bias_eos.T)
    sim_mix_mix = (z_mix_eos[:bias_len] @ z_mix_eos[bias_len:].T)
    sim_mix_mix_wrong = sim_mix_mix[~ind_]
    sim_mix_mix0_wrong = (z_mix_eos[:bias_len] @ z0_mix_eos[bias_len:].T)[~ind_]
    sim_mix0_mix_wrong = (z0_mix_eos[:bias_len] @ z_mix_eos[bias_len:].T)[~ind_]

    toy = 1
    thresh = 0.88
    if toy == 1:
        # match3 += torch.relu(max(sim_tar_mix0_wrong.max(), sim_tar0_mix_wrong.max(), sim_mix0_bias0_wrong.max()) # sim_tar0_mix_wrong.max()
        #                      - min(sim_tar_mix0_right.min(), sim_tar0_mix_right.min(), sim_mix0_bias0_right.min())) # sim_mix0_bias0_right.min()
        # match3 += torch.relu(sim_tar_mix0_wrong.max() - sim_mix0_bias0_right.min())
        # match3 += torch.relu(sim_tar0_mix_wrong.max() - sim_mix0_bias0_right.min())
        # match3 += torch.relu(sim_tar_mix0_wrong.max() - sim_mix0_bias0_right.min())
        pass
        # match3 += torch.relu(max(sim_mix_mix_wrong.max(), sim_mix0_mix_wrong.max(), sim_mix_mix0_wrong.max())
        #                      - min(sim_tar_mix0_right.min(), sim_tar0_mix_right.min(), sim_tar_mix_right.min()))
        # match3 += torch.relu(sim_mix0_mix0_right.mean() - sim_mix_mix_right.mean())
        # match3 += torch.relu(sim_tar_mix0_right.min() - sim_tar_mix0_wrong.max())
        # match3 += torch.mean((sim_tar_bias0 - sim_tar_bias0.max()) ** 2)
        # match3 += torch.relu(max(sim_mix_mix_wrong.max(), sim_mix0_mix_wrong.max(), sim_mix_mix0_wrong.max()) - min(
        #     sim_tar_mix0_right.min(), sim_tar0_mix_right.min(), sim_tar_mix_right.min()))
        # match3 += torch.relu(sim_mix0_mix0_right.mean() - sim_mix_mix_right.mean())
        # match3 += torch.relu(sim_tar0_mix0_right.mean() - sim_tar_mix0_right.mean())
        # match3 += torch.relu(sim_tar_mix0_right.min() - sim_tar_mix0_wrong.max())
        # bias_sim = torch.max(torch.tensor(thresh), sim_tar_bias0.mean())
        # if bias_sim > thresh:
        #     match3 += torch.max(torch.tensor(thresh), sim_tar_bias0.mean())
        match3 += torch.relu(0.8 - sim_tar0_bias0.mean())

    target_loss += match3
    # equalization_loss += abs(F.mse_loss(z_tar_eos[0] @ z0_mix_eos[0].T, z_tar_eos[0] @ z0_mix_eos[1].T).to(device))
    # # equalization_loss += abs(F.mse_loss(z_tar_eos[1] @ z0_mix_eos[0].T, z_tar_eos[1] @ z0_mix_eos[1].T).to(device))
    # # equalization_loss += abs(F.mse_loss(z_tar_eos[0] @ z0_mix_eos[2].T, z_tar_eos[0] @ z0_mix_eos[3].T).to(device))
    # equalization_loss += abs(F.mse_loss(z_tar_eos[1] @ z0_mix_eos[2].T, z_tar_eos[1] @ z0_mix_eos[3].T).to(device))
    # target_loss += equalization_loss
    from itertools import combinations
    #
    cnt = 0
    for (i, (tem1, re1, bias1)), (j, (tem2, re2, bias2)) in combinations(index_mapping.items(), 2):
        if re1 == re2:
            equalization_loss += sum(
                abs(F.mse_loss(z_tar_eos[k] @ z0_mix_eos[i].T, z_tar_eos[k] @ z0_mix_eos[j].T))
                for k in range(z_tar_eos.shape[0])
            )
            cnt += 1
    target_loss += equalization_loss / cnt * 5

    la_bird_bala = torch.mean((sim_tar_mix0[indices_1] - sim_tar_mix0[indices_1].mean()) ** 2)
    wa_bird_bala = torch.mean((sim_tar_mix0[indices_2] - sim_tar_mix0[indices_2].mean()) ** 2)
    mix_bird_bala = torch.mean((sim_tar_mix0[indices_3] - sim_tar_mix0[indices_3].mean()) ** 2)
    target_loss += la_bird_bala + wa_bird_bala # + mix_bird_bala
    # target_loss += match + match2 + (la_bird_bala + wa_bird_bala) * 20
    # target_loss += match3 + (la_bird_bala + wa_bird_bala) * 10  # + match2 # + match
    # target_loss += match
    '''
    print(f'{epoch}epoch |' 'distil_loss : ',
          round(distil_loss.item(), 4), 'target_loss : ', round(target_loss.item(), 4),
          'match1 : ', round(match.item(), 3),
          'match2 : ', round(match2.item(), 3),
          'match3 : ', round(match3.item(), 3),
          )
    '''
    loss = (1 - alpha) * distil_loss + alpha * target_loss
    loss.backward()
    optimizer.step()

owl_model.base_model.text_model.load_state_dict(target_model.text_encoder.text_model.state_dict())

text_queries = ["a doctor"]
image_path = "./doctor.png"
image = Image.open(image_path).convert('RGB')
inputs = processor(text=text_queries, images=image, return_tensors="pt")
outputs = owl_model(**inputs)

target_sizes = torch.tensor([image.size[::-1]])
results = processor.post_process_object_detection(outputs=outputs, target_sizes=target_sizes)

# Define confidence threshold
confidence_threshold = 0.1

fig, ax = plt.subplots(1, 1, figsize=(12, 8))
ax.imshow(image)

for i, result in enumerate(results):
    boxes = result["boxes"]
    scores = result["scores"]
    labels = result["labels"]

    high_confidence_indices = scores >= confidence_threshold
    filtered_boxes = boxes[high_confidence_indices]
    filtered_scores = scores[high_confidence_indices]
    filtered_labels = [labels[j] for j in range(len(labels)) if scores[j] >= confidence_threshold]

    for box, score, label in zip(filtered_boxes, filtered_scores, filtered_labels):
        box = box.detach().numpy()
        score = score.detach().numpy()
        label = text_queries[label]

        ax.add_patch(
            Rectangle((box[0], box[1]), box[2] - box[0], box[3] - box[1], fill=False, color="red", linewidth=2))
        ax.text(box[0], box[1], f"{label}: {score:.2f}", bbox=dict(facecolor='yellow', alpha=0.5))

plt.axis('off')  # Turn off axis numbers and ticks
plt.tight_layout()  # Make sure the annotations fit within the figure

# Save the annotated image to a file
output_path = "./annotated_doctor_.png"
plt.savefig(output_path, bbox_inches='tight', pad_inches=0)
plt.close()  # Close the figure to free up memory

# 텍스트 출력
for i, result in enumerate(results):
    print(f"Query: '{text_queries[i]}'")
    boxes = result["boxes"]
    scores = result["scores"]

    for box, score in zip(boxes, scores):
        print(f"  Box: {box.tolist()} - Score: {score:.4f}")
